In [7]:
from typing import List, Optional
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm.notebook import tqdm

import re
import os

from functools import partial
from scipy.stats import mode

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
# import optuna

from View import MyClass
mc = MyClass()

In [2]:
print(mc.__doc__)

Визуализация и предобработка данных
# Первичный обзор работа с памятью
* Метод "view_csv" принимает DataFrame и выводит данные о нем, переводит название столбцов в lower
возвращает data.columns, categorical, numerical
* Метод optim_memory оптимизирует память занимаемую данными, принимает DataFrame, возвращает DataFrame

# Статистика
* Метод "stat_frame" принимает DataFrame и возвращает DataFrame со статистиками
* Метод "corr" принимает DataFrame и target : str
* Метод corr_sign принимает data: DataFrame, drop_sign:List ( корреляция между признаками)
* Метод nonlianer_comun принимает data: DataFrame, numerical: list(список числовых признаков
target: str(по умолчанию "class"), num_sign: int(кол-во столбцов с нелинейной связью)
* Метод "percentile_99_1" принимает data: DataFrame, data_stat : DataFrame( возвращает метод "stat_frame"), flag: bool(
если флаг равен True возвращает фрейм где максимум заменен на 99 перцентиль, а минимум на 1 перцентиль)
по умолчанию flag = False
возвращат данны

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

print('The dimension of the train dataset is:', train.shape)
print('The dimension of the test dataset is:', test.shape)

The dimension of the train dataset is: (159256, 24)
The dimension of the test dataset is: (106171, 23)


In [4]:
train.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.8,19,13,17,0,1


In [6]:
train = train.drop("id", axis=1)
test = test.drop("id", axis=1)

In [6]:
train.columns, cat, num = mc.view_csv(train)
test.columns, cat_t, num_t = mc.view_csv(test)

rows: 159256, cols: 23
Всего категориальных признаков: 0
Index([], dtype='object')
Всего числовых признаков: 23
['systolic', 'height(cm)', 'hearing(left)', 'hdl', 'relaxation', 'gtp', 'hemoglobin', 'eyesight(left)', 'ast', 'urine protein', 'eyesight(right)', 'triglyceride', 'fasting blood sugar', 'smoking', 'serum creatinine', 'ldl', 'weight(kg)', 'hearing(right)', 'age', 'waist(cm)', 'cholesterol', 'dental caries', 'alt']
В данных пропусков нету
rows: 106171, cols: 22
Всего категориальных признаков: 0
Index([], dtype='object')
Всего числовых признаков: 22
['systolic', 'height(cm)', 'hearing(left)', 'hdl', 'relaxation', 'gtp', 'hemoglobin', 'eyesight(left)', 'ast', 'urine protein', 'eyesight(right)', 'triglyceride', 'fasting blood sugar', 'serum creatinine', 'ldl', 'weight(kg)', 'hearing(right)', 'age', 'waist(cm)', 'cholesterol', 'dental caries', 'alt']
В данных пропусков нету


In [9]:
fig,axes = plt.subplots(23,2,figsize=(15, 60),dpi=300)

for ind,col in enumerate(train.columns):
    if train[col].nunique()!=2:
        plt.subplot(23,2,2*ind+1)
        sns.histplot(train[col],bins=15,kde=True)
        plt.gca().set_title(col)
    elif col!="smoking":
        plt.subplot(23,2,2*ind+1)
        sns.countplot(data = train,x=col,hue="smoking")
        plt.gca().set_title(col)
    else:
        plt.subplot(23,2,2*ind+1)
        sns.countplot(data = train,x=col)
        plt.gca().set_title(col)
    
    if train[col].nunique()!=2:
        plt.subplot(23,2,2*ind+2)
        sns.boxplot(train[col],orient="h",palette="BuGn")
        plt.gca().set_title(col)
    else:
        plt.subplot(23,2,2*ind+2)
        sns.histplot(binwidth=0.5, x="dental caries", hue="smoking", data=train, stat="count", multiple="stack",palette="BuGn")
        plt.gca().set_title(col)

fig.tight_layout()
plt.show()

c:\program files\python37\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
c:\program files\python37\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
c:\program files\python37\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
c:\program files\python37\lib\s

# autoML

In [1]:
# !pip install autogluon > nul

# from autogluon.tabular import TabularDataset, TabularPredictor

# automl = TabularPredictor(
#     problem_type='binary',
#     label='smoking',
#     eval_metric='roc_auc'
# )

# automl.fit(train, presets='best_quality')
# automl.leaderboard()

# predictions = automl.predict(test)